In [5]:
# Use openai to provide answers with explanations that will be use to populate tha chat column and the answer column
# import the packages necessary to use openai
import openai
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [6]:
def BasicGeneration(userPrompt):
    completion = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = [
            {"role": "user", "content": userPrompt}
        ]
    )
    return completion.choices[0].message.content

In [7]:
options = pd.read_csv('options.csv')
options.head()

,Option,Topic,Unnamed: 2
0,beclo,"The topic of "" beclo"" cannot be identified bas...",NaN
1,beclomethasone inhaled,The topic of beclomethasone inhaled is 'Wheeze...,NaN
2,bed rest,The topic of bed rest is not included in the g...,NaN
3,bed rest 2 weeks,The topic of bed rest 2 weeks is not included ...,NaN
4,behavior training,The topic of behavior training is not listed i...,NaN


In [8]:
with open('plabtopics.csv', 'r') as f:
    topics = f.readlines()
    topics = [x.strip() for x in topics]
print(topics)

['Abdominal mass', 'Abdominal pain', 'Abnormal blood film', 'Abnormal blood sugar', 'Abnormalities of the urine', 'Abnormalities of the urine', 'Acid-base imbalance and blood gas abnormalities', 'Alcohol and drug use disorder and dependence', 'Anorexia and weight loss', 'Anxiety', 'Back and neck problems', 'Bites and stings', 'Blackouts and faints (funny turns)', 'Breast lump and/or pain', 'Breathlessness', 'Bruising/ bleeding/ purpura', 'Bullous', 'Cervical smear/ colposcopy', 'Chest pain', 'Chest pain', 'Clinical pharmacology', 'Clinical pharmacology', 'Collapse', 'Confusion', 'Confusion', 'Congenital abnormalities', 'Connective tissue disorders', 'Contraception', 'Cough and haemoptysis', 'Cranial nerve problems', 'Deliberate self-harm', 'Dermatological manifestations of systemic disease', 'Developmental problems', 'Developmental problems', 'Drawing from GMC explanatory guidance', 'Drawing from Good medical practice (2013) Duties of a doctor', 'Earache', 'Eating problems', 'Electroly

In [10]:
import time

def get_topic(working_df):
    start_time = time.time()
    failed_counter = 0
    for i in range(len(working_df)):
        
        if pd.isnull(working_df['Topic'][i]):
            try:
                print(f'Asking question number {i+1}')
                question_start_time = time.time()
                working_df['Topic'][i] = BasicGeneration(f"Create a list of presentations for {working_df['Option'][i]} based on the following list of presentations {topics}")
                elapsed_question_time = time.time() - question_start_time
                working_df.to_csv('options.csv', index=False)
                if elapsed_question_time > TIMEOUT_THRESHOLD_QUESTION:
                    print(f'Question number {i+1} took too long. Proceeding to the next question...')
                    failed_counter += 1
                    continue
                print(working_df['Topic'][i])
            except:
                print(f'Question number {i+1} failed')
                failed_counter += 1
                continue
        elapsed_total_time = time.time() - start_time
        if failed_counter > 10:
            print(f'Failed too many times. Saving progress and exiting...')
            working_df.to_csv('options.csv', index=False)
            return working_df
        if elapsed_total_time > TIMEOUT_THRESHOLD_TOTAL:
            print(f'Total time reached 30 minutes. Saving progress and exiting...')
            working_df.to_csv('options.csv', index=False)
            return working_df
       
    # save the working_df to plab_chatgpt.csv
    working_df.to_csv('options.csv', index=False)
    return working_df

TIMEOUT_THRESHOLD_QUESTION = 60  # 1 minute per question
TIMEOUT_THRESHOLD_TOTAL = 30 * 60  # 30 minutes in seconds
working_df = get_topic(options)


Asking question number 206
Presentations for calcium:

1. Abdominal mass
2. Abdominal pain
3. Abnormal blood film
4. Abnormal blood sugar
5. Abnormalities of the urine
6. Acid-base imbalance and blood gas abnormalities
7. Alcohol and drug use disorder and dependence
8. Anorexia and weight loss
9. Anxiety
10. Back and neck problems
11. Bites and stings
12. Blackouts and faints (funny turns)
13. Breast lump and/or pain
14. Breathlessness
15. Bruising/ bleeding/ purpura
16. Bullous
17. Cervical smear/ colposcopy
18. Chest pain
19. Clinical pharmacology
20. Collapse
21. Confusion
22. Congenital abnormalities
23. Connective tissue disorders
24. Contraception
25. Cough and haemoptysis
26. Cranial nerve problems
27. Deliberate self-harm
28. Dermatological manifestations of systemic disease
29. Developmental problems
30. Drawing from GMC explanatory guidance
31. Drawing from Good medical practice (2013) Duties of a doctor
32. Earache
33. Eating problems
34. Electrolyte abnormalities
35. Extrem